In [2]:
#Import the data

import pandas as pd
online = pd.read_csv('online.csv')
print(online.head())

   Unnamed: 0  InvoiceNo StockCode                      Description  Quantity  \
0      416792     572558     22745       POPPY'S PLAYHOUSE BEDROOM          6   
1      482904     577485     23196    VINTAGE LEAF MAGNETIC NOTEPAD         1   
2      263743     560034     23299     FOOD COVER WITH BEADS SET 2          6   
3      495549     578307    72349B  SET/6 PURPLE BUTTERFLY T-LIGHTS         1   
4      204384     554656     21756         BATH BUILDING BLOCK WORD         3   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2011-10-25 08:26:00       2.10       14286  United Kingdom  
1  2011-11-20 11:56:00       1.45       16360  United Kingdom  
2  2011-07-14 13:35:00       3.75       13933  United Kingdom  
3  2011-11-23 15:53:00       2.10       17290  United Kingdom  
4  2011-05-25 13:36:00       5.95       17663  United Kingdom  


In [3]:
#remove the time from InvoiceDate

import datetime as dt
def get_day(x): return dt.datetime(x.year,x.month,x.day)
cdate = pd.to_datetime(online['InvoiceDate'])
online['InvoiceDay'] = cdate.apply(get_day)
print(online)

       Unnamed: 0  InvoiceNo StockCode                         Description  \
0          416792     572558     22745          POPPY'S PLAYHOUSE BEDROOM    
1          482904     577485     23196       VINTAGE LEAF MAGNETIC NOTEPAD   
2          263743     560034     23299        FOOD COVER WITH BEADS SET 2    
3          495549     578307    72349B     SET/6 PURPLE BUTTERFLY T-LIGHTS   
4          204384     554656     21756            BATH BUILDING BLOCK WORD   
...           ...        ...       ...                                 ...   
70859      448575     575063     22804   PINK HANGING HEART T-LIGHT HOLDER   
70860       19970     537963     21733    RED HANGING HEART T-LIGHT HOLDER   
70861      264154     560089     23238   SET OF 4 KNICK KNACK TINS LONDON    
70862      199822     554103    85123A  WHITE HANGING HEART T-LIGHT HOLDER   
70863      312243     564342     22910   PAPER CHAIN KIT VINTAGE CHRISTMAS   

       Quantity          InvoiceDate  UnitPrice  CustomerID    

In [4]:
grouping = online.groupby('CustomerID')['InvoiceDay']
print(grouping.head())

0       2011-10-25
1       2011-11-20
2       2011-07-14
3       2011-11-23
4       2011-05-25
           ...    
70816   2011-10-14
70817   2011-12-05
70848   2011-02-23
70857   2011-12-06
70860   2010-12-09
Name: InvoiceDay, Length: 15619, dtype: datetime64[ns]


In [5]:
online['CohortDay'] = grouping.transform('min')
print(online.head())

   Unnamed: 0  InvoiceNo StockCode                      Description  Quantity  \
0      416792     572558     22745       POPPY'S PLAYHOUSE BEDROOM          6   
1      482904     577485     23196    VINTAGE LEAF MAGNETIC NOTEPAD         1   
2      263743     560034     23299     FOOD COVER WITH BEADS SET 2          6   
3      495549     578307    72349B  SET/6 PURPLE BUTTERFLY T-LIGHTS         1   
4      204384     554656     21756         BATH BUILDING BLOCK WORD         3   

           InvoiceDate  UnitPrice  CustomerID         Country InvoiceDay  \
0  2011-10-25 08:26:00       2.10       14286  United Kingdom 2011-10-25   
1  2011-11-20 11:56:00       1.45       16360  United Kingdom 2011-11-20   
2  2011-07-14 13:35:00       3.75       13933  United Kingdom 2011-07-14   
3  2011-11-23 15:53:00       2.10       17290  United Kingdom 2011-11-23   
4  2011-05-25 13:36:00       5.95       17663  United Kingdom 2011-05-25   

   CohortDay  
0 2011-04-11  
1 2011-09-12  
2 2011-07-1

In [6]:
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day
    
invoice_year, invoice_month, invoice_day = get_date_int(online, 'InvoiceDay')

cohort_year, cohort_month, cohort_day = get_date_int(online, 'CohortDay')

years_diff = invoice_year - cohort_year
months_diff = invoice_month - cohort_month
days_diff = invoice_day - cohort_day

online['CohortIndex'] = years_diff * 365 + months_diff * 30 + days_diff + 1
print(online.head())


   Unnamed: 0  InvoiceNo StockCode                      Description  Quantity  \
0      416792     572558     22745       POPPY'S PLAYHOUSE BEDROOM          6   
1      482904     577485     23196    VINTAGE LEAF MAGNETIC NOTEPAD         1   
2      263743     560034     23299     FOOD COVER WITH BEADS SET 2          6   
3      495549     578307    72349B  SET/6 PURPLE BUTTERFLY T-LIGHTS         1   
4      204384     554656     21756         BATH BUILDING BLOCK WORD         3   

           InvoiceDate  UnitPrice  CustomerID         Country InvoiceDay  \
0  2011-10-25 08:26:00       2.10       14286  United Kingdom 2011-10-25   
1  2011-11-20 11:56:00       1.45       16360  United Kingdom 2011-11-20   
2  2011-07-14 13:35:00       3.75       13933  United Kingdom 2011-07-14   
3  2011-11-23 15:53:00       2.10       17290  United Kingdom 2011-11-23   
4  2011-05-25 13:36:00       5.95       17663  United Kingdom 2011-05-25   

   CohortDay  CohortIndex  
0 2011-04-11          195  


In [16]:
cohort_data = online.groupby('CustomerID').apply(pd.Series.nunique).reset_index()
cohort_counts = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')

AttributeError: 'DataFrame' object has no attribute 'unique'